In [ ]:
# import all the libraries needed for this project
import numpy as np
import pandas as pd
import math
from math import sqrt
from numpy import mean
from numpy.random import rand
from numpy.random import randn
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense , Activation, Dropout, CategoryEncoding
from keras.optimizers import Adam ,RMSprop
from keras import  backend as K
#from tensorflow.keras.utils import to_categorical

In [ ]:
train=pd.read_csv("/content/train.csv")
test_x=pd.read_csv("/content/test.csv")
test_y=pd.read_csv("/content/sample_submission.csv")

In [ ]:
from sklearn.model_selection import train_test_split
test_y=test_y.drop(["Id"],axis=1)
test_x=test_x.drop(["Id"],axis=1)
train=train.drop(["Id"],axis=1)

In [ ]:
test_x["SalePrice"]=test_y
df=pd.concat([test_x,train])
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,169277.052498
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,187758.393989
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,183583.683570
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,179317.477511
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,150730.079977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000.000000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000.000000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500.000000
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125.000000


In [ ]:
Y=df["SalePrice"]
X=df.drop(["SalePrice"],axis=1)

In [ ]:
#getting list of columns with Null Values again
print(round(df.isnull().sum()/len(df.index)*100,2).sort_values(ascending=False)[ round(df.isnull().sum()/len(df.index),2) > 0 ] )

PoolQC          99.66
MiscFeature     96.40
Alley           93.22
Fence           80.44
MasVnrType      60.50
FireplaceQu     48.65
LotFrontage     16.65
GarageCond       5.45
GarageQual       5.45
GarageYrBlt      5.45
GarageFinish     5.45
GarageType       5.38
BsmtCond         2.81
BsmtExposure     2.81
BsmtQual         2.77
BsmtFinType2     2.74
BsmtFinType1     2.71
MasVnrArea       0.79
dtype: float64


In [ ]:
df['age']=df['YrSold']-df['YearBuilt']
df['garage_age'] = df['YrSold'] - df['GarageYrBlt']
df['remodel_age'] = df['YrSold'] - df['YearRemodAdd']

df.drop('GarageYrBlt',axis=1,inplace=True)

df[['MSSubClass']] = df[['MSSubClass']].astype(str)
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df['MoSold'].astype(str)

In [ ]:
#From the Data Description
#NA = No Pool for PoolQC
df['PoolQC'] = df['PoolQC'].fillna('None')
#NA = No Misc Feature for MiscFeature
df['MiscFeature'] = df['MiscFeature'].fillna('None')
#NA = No Alley
df['Alley'] = df['Alley'].fillna('None')
#NA = No Fireplace for FireplaceQu
df['FireplaceQu'] = df['FireplaceQu'].fillna('None')
#NA = No Fence
df['Fence'] = df['Fence'].fillna('None')
#NA = No Fireplace
df['FireplaceQu'] = df['FireplaceQu'].fillna('None')
#NA = No Garage
df['GarageCond'] = df['GarageCond'].fillna('None')
df['GarageType'] = df['GarageType'].fillna('None')
df['GarageFinish'] = df['GarageFinish'].fillna('None')
df['GarageQual'] = df['GarageQual'].fillna('None')
df['garage_age'] = df['GarageQual'].fillna(0)   #No Garage No Age
#NA = No Basement
df['BsmtExposure'] = df['BsmtExposure'].fillna('None')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('None')
df['BsmtCond'] = df['BsmtCond'].fillna('None')
df['BsmtQual'] = df['BsmtQual'].fillna('None')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('None')
#NA = No Masonery Area
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

In [ ]:
#LotFrontage : Replacing Null value with the median of the neighbourhood
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
# Filling the Electrical 0.06% rows with the mode
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2919 entries, 0 to 1459
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   object 
 1   MSZoning       2915 non-null   object 
 2   LotFrontage    2919 non-null   float64
 3   LotArea        2919 non-null   int64  
 4   Street         2919 non-null   object 
 5   Alley          2919 non-null   object 
 6   LotShape       2919 non-null   object 
 7   LandContour    2919 non-null   object 
 8   Utilities      2917 non-null   object 
 9   LotConfig      2919 non-null   object 
 10  LandSlope      2919 non-null   object 
 11  Neighborhood   2919 non-null   object 
 12  Condition1     2919 non-null   object 
 13  Condition2     2919 non-null   object 
 14  BldgType       2919 non-null   object 
 15  HouseStyle     2919 non-null   object 
 16  OverallQual    2919 non-null   int64  
 17  OverallCond    2919 non-null   int64  
 18  YearBuilt    

In [ ]:
round(df.isnull().sum()/len(df.index)*100,5).sort_values(ascending=False)

MSZoning        0.13703
BsmtFullBath    0.06852
BsmtHalfBath    0.06852
Utilities       0.06852
Functional      0.06852
                 ...   
ExterCond       0.00000
ExterQual       0.00000
MasVnrArea      0.00000
MasVnrType      0.00000
remodel_age     0.00000
Length: 82, dtype: float64

In [ ]:
print(df['Utilities'].value_counts())
print(df['Street'].value_counts())
print(df['PoolQC'].value_counts())

Utilities
AllPub    2916
NoSeWa       1
Name: count, dtype: int64
Street
Pave    2907
Grvl      12
Name: count, dtype: int64
PoolQC
None    2909
Ex         4
Gd         4
Fa         2
Name: count, dtype: int64


In [ ]:
# Trying to remove redundant features
def redundant_feature(df):
    redundant = []
    for i in df.columns:
        counts = df[i].value_counts()
        count_max = counts.iloc[0]
        if count_max / len(df) * 100 > 99:
            redundant.append(i)
    redundant = list(redundant)
    return redundant

In [ ]:
Y=df["SalePrice"]
df=df.drop(["SalePrice"],axis=1)

In [ ]:
#obtain all the numerical features
numerical_features = df.dtypes[df.dtypes != 'object'].index
non_numerical_features= df.dtypes[df.dtypes=="object"].index
#train_x[non_numerical_features]=to_categorical(train_x[non_numerical_features])
#apply standardization to each feature
df=df.fillna(0)
df[numerical_features] = df[numerical_features].apply( lambda x: (x-x.mean()) / x.std())
encoded_train = pd.get_dummies(df, columns=non_numerical_features, dtype=float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(encoded_train, Y, test_size=0.15, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential,layers
from keras.layers import BatchNormalization

In [ ]:
#Min max for test and train data
y_min=np.min(y_train)
y_max=np.max(y_train)
y_train=(y_train- y_min)/ (y_max - y_min)

y_min=np.min(y_test)
y_max=np.max(y_test)
y_test=(y_test- y_min)/ (y_max - y_min)

In [ ]:
X_train.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,garage_age_Ex,garage_age_Fa,garage_age_Gd,garage_age_None,garage_age_Po,garage_age_TA
1025,0.527893,0.508037,-0.772420,-0.507197,-0.505515,0.657380,-0.566919,-0.121315,2.792512,-0.519983,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
373,0.435741,0.059070,-0.772420,0.391170,-0.604553,-1.496313,-0.566919,-0.029130,0.770986,-1.275231,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
485,-0.255399,2.669189,-0.063174,-3.202298,-0.637566,-1.544173,-0.566919,-0.968539,-0.292975,2.437312,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1454,-0.347551,-0.338293,0.646073,-0.507197,1.079091,0.992399,-0.566919,-0.068638,-0.292975,0.569666,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
927,0.159285,0.092670,-1.481667,-0.507197,-0.241414,-0.969855,-0.566919,-0.968539,-0.292975,-1.275231,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
from keras.layers import Input
model = Sequential(
    [
        Dense(units=400, activation='linear', input_dim=346),
        BatchNormalization(),
        Dropout(0.05),
        Dense(500, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)


In [ ]:
from keras.losses import BinaryCrossentropy
model.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])


In [ ]:
xtrain, xval, ytrain, yval = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [ ]:
model.fit(xtrain, ytrain, epochs=50,batch_size = 64,
                  validation_data=(xval, yval))

Epoch 1/50
33/33 [==============================] - 3s 17ms/step - loss: 1.5878 - mean_squared_error: 1.5878 - val_loss: 0.0557 - val_mean_squared_error: 0.0557
Epoch 2/50
33/33 [==============================] - 0s 9ms/step - loss: 0.4838 - mean_squared_error: 0.4838 - val_loss: 0.0449 - val_mean_squared_error: 0.0449
Epoch 3/50
33/33 [==============================] - 0s 9ms/step - loss: 0.3701 - mean_squared_error: 0.3701 - val_loss: 0.0887 - val_mean_squared_error: 0.0887
Epoch 4/50
33/33 [==============================] - 0s 9ms/step - loss: 0.5017 - mean_squared_error: 0.5017 - val_loss: 0.0863 - val_mean_squared_error: 0.0863
Epoch 5/50
33/33 [==============================] - 0s 9ms/step - loss: 0.2465 - mean_squared_error: 0.2465 - val_loss: 0.0637 - val_mean_squared_error: 0.0637
Epoch 6/50
33/33 [==============================] - 0s 8ms/step - loss: 0.1750 - mean_squared_error: 0.1750 - val_loss: 0.0651 - val_mean_squared_error: 0.0651
Epoch 7/50
33/33 [=====================

In [ ]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 4ms/step - loss: 0.0075 - mean_squared_error: 0.0075


[0.007494801189750433, 0.007494801189750433]

In [ ]:
model.compile(loss="mean_squared_error",
                  optimizer=RMSprop(learning_rate=0.01),
                  metrics=["mean_squared_error"])

In [ ]:
model.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))

Epoch 1/50
66/66 [==============================] - 2s 13ms/step - loss: 401.0031 - mean_squared_error: 401.0031 - val_loss: 44.9193 - val_mean_squared_error: 44.9193
Epoch 2/50
66/66 [==============================] - 1s 10ms/step - loss: 8.0085 - mean_squared_error: 8.0085 - val_loss: 1.7512 - val_mean_squared_error: 1.7512
Epoch 3/50
66/66 [==============================] - 1s 9ms/step - loss: 37.9896 - mean_squared_error: 37.9896 - val_loss: 0.1246 - val_mean_squared_error: 0.1246
Epoch 4/50
66/66 [==============================] - 1s 9ms/step - loss: 52.2371 - mean_squared_error: 52.2371 - val_loss: 5.7005 - val_mean_squared_error: 5.7005
Epoch 5/50
66/66 [==============================] - 1s 9ms/step - loss: 1.5964 - mean_squared_error: 1.5964 - val_loss: 0.5656 - val_mean_squared_error: 0.5656
Epoch 6/50
66/66 [==============================] - 1s 9ms/step - loss: 57.3726 - mean_squared_error: 57.3726 - val_loss: 0.0421 - val_mean_squared_error: 0.0421
Epoch 7/50
66/66 [========

In [ ]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00


[nan, 0.0]

14/14 [==============================] - 0s 3ms/step - loss: 2438689024.0000 - accuracy: 0.0000e+00


[2438689024.0, 0.0]